In [1]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv
import os
from PIL import ImageFont, ImageDraw, Image
import shutil

ImportError: No module named _caffe

In [2]:
input_dir = u'E:\\Diploma\\Inputs\\'
output_dir = u'E:\\Diploma\\Outputs\\'
cropped_dir = u'E:\\Diploma\\Outputs_cropped\\'

simhei = 'Simhei'
simsun = 'Simsun'

size = 60

In [3]:
def prepare_dir(dir_name, subdir_name):
    if os.path.exists(dir_name + subdir_name):
        shutil.rmtree(dir_name + subdir_name)
    
    os.makedirs(dir_name + subdir_name)

In [51]:
def draw_hieroglyph(font, filename, subdir):
    hieroglyph = Image.new('1', (size, size), color=1)
    draw = ImageDraw.Draw(hieroglyph)
    draw.text((0, 0), filename[0], font=font, fill=0)
    
    hieroglyph.save(output_dir + subdir + '\\' + filename, 'BMP')

In [52]:
prepare_dir(output_dir, simhei)
prepare_dir(output_dir, simsun)

font_simhei = ImageFont.truetype('simhei.ttf', size);
font_simsun = ImageFont.truetype('simsun.ttc', size);

for filename in os.listdir(input_dir):
    draw_hieroglyph(font_simhei, filename, simhei)
    draw_hieroglyph(font_simsun, filename, simsun)

Обрежем края на шрифтовых изображениях иероглифов

In [53]:
black_pixel = 0

def has_row_black(image, row):
    width = image.size[0]
    for x in range(0, width):
        if image.getpixel((x, row)) == black_pixel:
            return True
    return False    
    
def has_col_black(image, col):
    height = image.size[1]
    for y in range(0, height):
        if image.getpixel((col, y)) == black_pixel:
            return True
    return False

def crop_image(image):
    width, height = image.size
    
    left = 0
    upper = 0
    right = 0
    lower = 0
    
    for y in range(0, height):
        if has_row_black(image, y):
            upper = y
            break
        
    for x in range(0, width):
        if has_col_black(image, x):
            left = x
            break
    
    for x in range(width - 1, -1, -1):
        if has_col_black(image, x):
            right = x
            break
        
    for y in range(height - 1, -1, -1):
        if has_row_black(image, y):
            lower = y
            break
                
    return image.crop((left,upper,right,lower))

In [54]:
prepare_dir(cropped_dir, simhei)
prepare_dir(cropped_dir, simsun)

font_names = [simhei, simsun]

for font_name in font_names:
    for filename in os.listdir(output_dir + font_name):
        image = Image.open(output_dir + font_name + '\\' + filename)
        cropped_image = crop_image(image)
        cropped_image.save(cropped_dir + font_name + '\\' + filename, 'BMP')

In [32]:
glyph_shapes = []
font_simhei_shapes = []
font_simsun_shapes = []

iteration = 0

for filename in os.listdir(input_dir):
    glyph_image = Image.open(input_dir + filename)
    font_simhei_image = Image.open(cropped_dir + simhei + '\\' + filename)
    font_simsun_image = Image.open(cropped_dir + simsun + '\\' + filename)
    
    glyph_shapes.append(gliff_image.size)
    font_simhei_shapes.append(font_simhei_image.size)
    font_simsun_shapes.append(font_simsun_image.size)

In [9]:
print glyph_shapes
print font_simhei_shapes
print font_simsun_shapes

[(42, 46), (42, 46), (43, 47), (58, 54), (44, 41), (41, 41), (45, 41), (43, 41), (38, 44), (44, 42)]
[(37, 41), (37, 41), (37, 41), (29, 29), (53, 49), (53, 49), (53, 50), (53, 50), (51, 48), (51, 48)]
[(36, 42), (36, 42), (36, 42), (33, 31), (51, 51), (51, 51), (54, 50), (54, 50), (51, 50), (51, 50)]


In [33]:
def get_ratios(shapes):
    return [float(width) / height for (width, height) in shapes]

In [34]:
glyph_ratios = get_ratios(glyph_shapes)
font_simhei_ratios = get_ratios(font_simhei_shapes)
font_simsun_ratios = get_ratios(font_simsun_shapes)

In [35]:
print len(glyph_ratios), len(font_simhei_ratios), len(font_simsun_ratios)

25795 25795 25795


In [38]:
glyph_simhei = [abs(x - y) for (x,y) in zip(glyph_ratios, font_simhei_ratios)]
glyph_simsun = [abs(x - y) for (x,y) in zip(glyph_ratios, font_simsun_ratios)]

In [39]:
print sum(glyph_simhei) / len(glyph_simhei)
print sum(glyph_simsun) / len(glyph_simsun)

0.0273315362265
0.0319066214754


In [42]:
print min(font_simsun_shapes)

(5, 54)


In [62]:
network_inputs_dir = u'E:\\Diploma\\Network_inputs\\'
network_outputs_dir = u'E:\\Diploma\\Network_outputs\\'
default_size = 23

In [69]:
def centre_images(input_dir, output_dir):
    
    iteration = 0

    for filename in os.listdir(input_dir):
        image = Image.open(input_dir + filename)
        image_background = Image.new('1', (default_size, default_size), color=1)
        width, height = image.size
        new_width = default_size
        new_height = default_size

        left = 0
        upper = 0

        if width >= height:
            new_height = int(round(float(height) / width * default_size))
            upper = int(round((default_size - new_height) / 2.0))
        else:
            new_width = int(round(float(width) / height * default_size))
            left = int(round((default_size - new_width) / 2.0))

        resized_image = image.resize((new_width, new_height))
        image_background.paste(resized_image, (left, upper))        
        image_background.save(output_dir + filename, 'BMP')

In [70]:
centre_images(input_dir, network_inputs_dir)
print 'originals processed'
centre_images(cropped_dir + simhei + '\\', network_outputs_dir + simhei + '\\')
print 'simhei processed'
centre_images(cropped_dir + simsun + '\\', network_outputs_dir + simsun + '\\')
print 'simsun processed'

originals processed
simhei processed
simsun processed
